In [48]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_parquet('sample_txn_data_2022.parquet', engine='pyarrow')

In [24]:
df.head(20)

,dw_gc_header,business_date,fiscal_week_end,fiscal_week,fiscal_year,daypart_name,lineitem_sequence,lineitem_seq_parent,lineitem_description,parent_product_description,...,actmodqty,actdiscqty,actprodprice,actgrosssales,actnetsales,actpromosales,actdiscsales,acttax,dw_gc_header_2,token_primary_account_identifier
1,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,1,1,NON-COMBO-ITEM,Beefy 5-Layer Burrito,...,0.0,0,2.99,2.99,2.99,0.00,0.00,0.00,42188263563,0880344659239910
2,42188263563,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,2,2,NON-COMBO-ITEM,Cheesy Bean and Rice Burrito,...,0.0,0,1.29,1.29,1.29,0.00,0.00,0.00,42188263563,0880344659239910
4,42201144646,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,BREAKFAST,1,1,NON-COMBO-ITEM,Breakfast Crunchwrap - Bacon,...,0.0,0,3.49,3.49,3.49,0.00,0.00,0.00,42201144646,8819419820104328
6,42197843459,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,LUNCH,1,1,NON-COMBO-ITEM,Cheesy Fiesta Potatoes,...,0.0,0,1.89,1.89,1.89,0.00,0.00,0.00,42197843459,8456488655739252
8,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,1,1,NON-COMBO-ITEM,Chicken Quesadilla,...,0.0,0,4.99,14.97,14.97,0.00,0.00,0.00,42188307845,8504411087099781
9,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,2,2,NON-COMBO-ITEM,Soft Taco,...,0.0,0,1.89,1.89,1.89,0.00,0.00,0.00,42188307845,8504411087099781
10,42188307845,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,AFTERNOON,3,3,NON-COMBO-ITEM,NO Sauce Packet,...,0.0,0,0.00,0.00,0.00,0.00,0.00,0.00,42188307845,8504411087099781
12,42185146985,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,4,4,NON-COMBO-ITEM,Bean Burrito,...,0.0,0,1.49,1.49,1.49,0.00,0.00,0.00,42185146985,0761848206682093
13,42185146985,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,5,5,NON-COMBO-ITEM,Timed Test Item,...,0.0,0,1.49,1.49,1.49,0.00,0.00,0.00,42185146985,0761848206682093
14,42185146985,2021-12-31,2022-01-04,Y2022 Q01 P01 W01,Y2022,EVENING,3,3,NON-COMBO-ITEM,Large Mountain Dew®,...,0.0,0,2.19,2.19,2.19,0.00,0.00,0.00,42185146985,0761848206682093


In [7]:
df = df[df["parent_product_description"].notna()]

In [41]:
transactions = df[["dw_gc_header","parent_product_description"]]

In [42]:
transactions.rename(columns = {'dw_gc_header':'order_id','parent_product_description':'product_name'}, inplace = True)

In [43]:
transactions.head()

,order_id,product_name
1,42188263563,Beefy 5-Layer Burrito
2,42188263563,Cheesy Bean and Rice Burrito
4,42201144646,Breakfast Crunchwrap - Bacon
6,42197843459,Cheesy Fiesta Potatoes
8,42188307845,Chicken Quesadilla


In [44]:
transactions = transactions.groupby('order_id')['product_name'].apply(list).values.tolist()

In [45]:
transactions

[['Large Mountain Dew Baja Blast®', 'Chicken Chipotle Melt'],
 ['Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'Cheesy Roll Up',
  'Large Sierra Mist®',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito'],
 ['NO Sauce Packet', 'Beefy Melt Burrito'],
 ['Crunchy Taco Supreme®',
  'Sml Drink',
  'Lrg Drink',
  'Power Menu Bowl - Chicken'],
 ['Nachos BellGrande®',
  'Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'NO Sauce Packet',
  'Small Mountain Dew®',
  'Small Pepsi®'],
 ['Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo'],
 ['Large Mountain Dew Baja Blast®'],
 ['Chips and Nacho Cheese Sauce', 'Fire Sauce Packet', 'Beefy Melt Burrito'],
 ['Timed Test Item',
  'Chicken Chipotle Melt',
  'Fiesta Veggie Burrito',
  'Beefy Melt Burrito'],
 [

In [82]:
sample = transactions[0:20]

In [56]:
sample

[['Large Mountain Dew Baja Blast®', 'Chicken Chipotle Melt'],
 ['Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'Cheesy Roll Up',
  'Large Sierra Mist®',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito',
  'Cheesy Bean and Rice Burrito'],
 ['NO Sauce Packet', 'Beefy Melt Burrito'],
 ['Crunchy Taco Supreme®',
  'Sml Drink',
  'Lrg Drink',
  'Power Menu Bowl - Chicken'],
 ['Nachos BellGrande®',
  'Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'Soft Taco',
  'NO Sauce Packet',
  'Small Mountain Dew®',
  'Small Pepsi®'],
 ['Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo',
  'Chipotle Cheddar Chalupa Combo'],
 ['Large Mountain Dew Baja Blast®'],
 ['Chips and Nacho Cheese Sauce', 'Fire Sauce Packet', 'Beefy Melt Burrito'],
 ['Timed Test Item',
  'Chicken Chipotle Melt',
  'Fiesta Veggie Burrito',
  'Beefy Melt Burrito'],
 [

In [85]:
for i in range(len(sample)):
    z = []
    for j in range(len(sample[i])):
        if sample[i][j] not in z:
            z.append(sample[i][j])
    sample[i] = z

In [86]:
transactions = sample
te = TransactionEncoder()
te_array = te.fit_transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

In [87]:
df_encoded

,2 Chicken Chalupas Supreme Combo,3 Soft Tacos Supreme® Combo,Bean Burrito,Beefy Melt Burrito,Black Bean Chalupa Supreme®,Burrito Supreme® - Seasoned Beef,Burrito Supreme® Combo,Chalupa Cravings Box,Chalupa Supreme® - Seasoned Beef,Cheesy Bean and Rice Burrito,...,NO Sauce Packet,Nacho Cheese Doritos® Locos Tacos,Nachos BellGrande®,Power Menu Bowl - Chicken,Small Mountain Dew®,Small Pepsi®,Sml Drink,Soft Taco,Soft Taco Supreme®,Timed Test Item
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,True,True,False,True,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True


In [88]:
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.10,(Bean Burrito)
1,0.20,(Beefy Melt Burrito)
2,0.10,(Cheesy Bean and Rice Burrito)
3,0.15,(Cheesy Roll Up)
4,0.10,(Chicken Chipotle Melt)
5,0.10,(Chips and Nacho Cheese Sauce)
6,0.10,(Crunchwrap Supreme® Combo)
7,0.10,(Fire Sauce Packet)
8,0.10,(Hot Sauce Packet)
9,0.15,(Large Mountain Dew Baja Blast®)


In [93]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [90]:
main_product = "Bean Burrito"
rules_filtered = rules[rules['antecedents'].apply(lambda x: main_product in x)]

In [91]:
rules_sorted = rules_filtered.sort_values(['confidence', 'support'], ascending=[False, False])


In [92]:
top_recommendations = rules_sorted.head(5)['consequents'].tolist()
print(f"Top side product recommendations for {main_product}: {top_recommendations}")

Top side product recommendations for Bean Burrito: [frozenset({'Mild Sauce Packet'})]


In [94]:
echo "# 2023-IP" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/WilliamCCCCCC/2023-IP.git
git push -u origin main

SyntaxError: invalid syntax (4016247671.py, line 1)